### Hoodnaming

This notebook converts the point names downloaded from WebCTRL into a list of pseudo point names for later use

In [4]:
import pandas as pd

df = pd.read_csv("pointnames.csv")
floor_map = {"ground":"Ground", "1st":"First","2nd":"Second", "3rd":"Third","4th":"Fourth"}
df["FloorNum"] = df["Floor"].map(floor_map)
display(df)

,Unnamed: 0,Lab,Floor,Server,Hood,Sash,Occupancy,Internal Temp,External Temp,FloorNum
0,0,g54,ground,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,Ground
1,1,141,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
2,2,143,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
3,3,147b,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
4,4,153,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
...,...,...,...,...,...,...,...,...,...,...
61,61,445b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
62,62,453,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
63,63,453b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
64,64,457,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth


In [5]:
#Biotech.floor_num.lab_num.hood_num.{pointType}
#4 Point Types:[energy.occ,energy.unocc,sashOpenTime.occ, sashOpenTime.unocc]

def point_maker(lab,floor,hood):
    nstr = "Biotech." + str(floor) + "Floor" + ".Lab_" +  str(lab)   + ".Hood_" + str(hood) + "."
    nlist = [nstr+"energy.occ",nstr+"energy.unocc",nstr+"sashOpenTime.occ",nstr+"sashOpenTime.unocc"]
    return nlist



#display(df)

# test_case = point_maker(141,1,1)
# test_case

def make_all_points(df):
    energy_occ = []
    energy_unocc = []
    sashOpenTime_occ = []
    sashOpenTime_unocc = []
    for i in range(len(df["FloorNum"])):
        response = point_maker(df["Lab"][i],df["FloorNum"][i],1)
        energy_occ.append(response[0])
        energy_unocc.append(response[1])
        sashOpenTime_occ.append(response[2])
        sashOpenTime_unocc.append(response[3])
    df["Energy Occupied"] = energy_occ
    df["Energy Unoccupied"] = energy_unocc
    df["Sash Open Time Occupied"] = sashOpenTime_occ
    df["Sash Open Time Unoccupied"] = sashOpenTime_unocc
    return df

ndf = make_all_points(df)
display(ndf)







,Unnamed: 0,Lab,Floor,Server,Hood,Sash,Occupancy,Internal Temp,External Temp,FloorNum,Energy Occupied,Energy Unoccupied,Sash Open Time Occupied,Sash Open Time Unoccupied
0,0,g54,ground,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,Ground,Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ,Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc,Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTim...,Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTim...
1,1,141,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_141.Hood_1.energy.occ,Biotech.FirstFloor.Lab_141.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_141.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_141.Hood_1.sashOpenTime...
2,2,143,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_143.Hood_1.energy.occ,Biotech.FirstFloor.Lab_143.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_143.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_143.Hood_1.sashOpenTime...
3,3,147b,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_147b.Hood_1.energy.occ,Biotech.FirstFloor.Lab_147b.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_147b.Hood_1.sashOpenTim...,Biotech.FirstFloor.Lab_147b.Hood_1.sashOpenTim...
4,4,153,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_153.Hood_1.energy.occ,Biotech.FirstFloor.Lab_153.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_153.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_153.Hood_1.sashOpenTime...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,445b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_445b.Hood_1.energy.occ,Biotech.FourthFloor.Lab_445b.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_445b.Hood_1.sashOpenTi...,Biotech.FourthFloor.Lab_445b.Hood_1.sashOpenTi...
62,62,453,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_453.Hood_1.energy.occ,Biotech.FourthFloor.Lab_453.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_453.Hood_1.sashOpenTim...,Biotech.FourthFloor.Lab_453.Hood_1.sashOpenTim...
63,63,453b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_453b.Hood_1.energy.occ,Biotech.FourthFloor.Lab_453b.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_453b.Hood_1.sashOpenTi...,Biotec

In [6]:
ndf.to_csv("namingconvention.csv")